# SW-1-Setup

**Navigation** : [Index](README.md) | [2-RDFBasics >>](SW-2-RDFBasics.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Situer le Web semantique dans l'histoire du Web et la vision de Tim Berners-Lee
2. Identifier les couches de la pile technologique du Web semantique (RDF, SPARQL, OWL, SHACL)
3. Installer et configurer la bibliotheque dotNetRDF dans un environnement .NET Interactive
4. Creer et afficher votre premier graphe RDF ("Hello World" semantique)

### Concepts cles

| Concept | Description |
|---------|-------------|
| Web semantique | Extension du Web ou l'information est comprehensible par les machines |
| RDF | Modele de donnees base sur des triplets (sujet-predicat-objet) |
| dotNetRDF | Bibliotheque C# open-source pour manipuler des donnees RDF |
| Triplet | Unite fondamentale de RDF : une assertion liant deux ressources |

### Prerequis
- .NET 9.0 avec .NET Interactive pour Jupyter
- Aucune connaissance prealable en Web semantique requise

### Duree estimee : 20 minutes

---

## 1. Contexte historique du Web semantique

### La vision de Tim Berners-Lee

En 2001, Tim Berners-Lee, inventeur du World Wide Web, publie avec James Hendler et Ora Lassila un article fondateur dans *Scientific American* intitule *"The Semantic Web"*. Sa vision : un Web ou les informations ne sont pas seulement lisibles par les humains, mais egalement **comprehensibles et exploitables par les machines**.

Le Web classique (Web 1.0/2.0) repose sur des documents HTML lies entre eux par des hyperliens. Le Web semantique ajoute une couche de **sens** en structurant les donnees selon des standards ouverts.

### La pile technologique du W3C ("Layer Cake")

Le W3C (World Wide Web Consortium) a defini une architecture en couches pour le Web semantique :

```
 +-----------------------------------------+
 |              Applications               |
 +-----------------------------------------+
 |          Trust / Proof / Logic           |
 +-----------------------------------------+
 |    OWL (Ontologies)  |  SHACL (Shapes)  |
 +-----------------------------------------+
 |         SPARQL (Requetes)                |
 +-----------------------------------------+
 |       RDFS (Schema / Vocabulaire)        |
 +-----------------------------------------+
 |         RDF (Modele de donnees)          |
 +-----------------------------------------+
 |     XML / JSON-LD / Turtle (Syntaxe)     |
 +-----------------------------------------+
 |          URI / IRI (Identifiants)        |
 +-----------------------------------------+
 |            Unicode (Encodage)            |
 +-----------------------------------------+
```

### Description des couches principales

| Couche | Standard | Role | Notebook(s) de la serie |
|--------|----------|------|-------------------------|
| **Identifiants** | URI / IRI | Nommer de facon unique chaque ressource | SW-2 |
| **Modele de donnees** | RDF | Representer l'information en triplets | SW-2, SW-3 |
| **Schema** | RDFS | Definir des classes et proprietes | SW-6 |
| **Requetes** | SPARQL | Interroger des graphes RDF | SW-5 |
| **Ontologies** | OWL | Raisonnement logique sur les donnees | SW-7 |
| **Contraintes** | SHACL | Valider la conformite des donnees | SW-8 |

> **Point cle** : Chaque couche s'appuie sur les precedentes. RDF est la fondation sur laquelle repose tout le reste.

### Pourquoi le Web semantique ?

Le Web semantique repond a plusieurs besoins concrets :

| Probleme du Web classique | Solution du Web semantique |
|---------------------------|----------------------------|
| Donnees en silos (chaque site a son format) | Standards ouverts (RDF, SPARQL) |
| Liens entre documents, pas entre donnees | Liens entre entites (Linked Data) |
| Recherche par mots-cles uniquement | Requetes structurees (SPARQL) |
| Pas de raisonnement automatique | Inference logique (OWL, RDFS) |

**Exemples d'applications reelles** :
- **Wikidata** : base de connaissances structuree de Wikipedia
- **DBpedia** : extraction automatique de donnees RDF depuis Wikipedia
- **Schema.org** : vocabulaire utilise par Google, Bing, Yahoo pour le SEO semantique
- **Graphes de connaissances** : Google Knowledge Graph, Amazon Product Graph

---

## 2. Installation de dotNetRDF

### Presentation de la bibliotheque

**dotNetRDF** est une bibliotheque open-source C# pour travailler avec RDF. Elle fournit :
- Manipulation de graphes RDF en memoire
- Lecture/ecriture dans 10+ formats de serialisation
- Moteur de requetes SPARQL
- Support pour les Triple Stores
- Integration avec des bases de donnees RDF (Fuseki, Blazegraph, etc.)

Installons le package NuGet dans notre environnement .NET Interactive.

In [ ]:
// Installation du package dotNetRDF (version epinglee pour reproductibilite)
#r "nuget: dotNetRDF, 3.2.1"

### Interpretation

La commande `#r "nuget: ..."` est specifique a .NET Interactive et permet d'installer des packages NuGet directement dans le notebook. L'installation telecharge la bibliotheque et ses dependances.

> **Note technique** : Nous epinglons la version `3.2.1` pour garantir la reproductibilite. Sans version explicite, le notebook pourrait se comporter differemment dans le futur si une nouvelle version introduit des changements incompatibles.

### Verification de l'installation

Verifions que les espaces de noms principaux de dotNetRDF sont accessibles.

In [ ]:
// Verification des espaces de noms principaux
using VDS.RDF;
using VDS.RDF.Parsing;
using VDS.RDF.Writing;
using System;

Console.WriteLine("dotNetRDF charge avec succes.");
Console.WriteLine($"Espaces de noms disponibles : VDS.RDF, VDS.RDF.Parsing, VDS.RDF.Writing");

### Interpretation

Si l'execution s'est deroulee sans erreur, les trois espaces de noms principaux sont prets :

| Espace de noms | Contenu | Usage |
|----------------|---------|-------|
| `VDS.RDF` | Classes de base (`Graph`, `Triple`, `INode`) | Creation et manipulation de graphes |
| `VDS.RDF.Parsing` | Parsers (`TurtleParser`, `RdfXmlParser`) | Lecture de fichiers RDF |
| `VDS.RDF.Writing` | Writers (`NTriplesWriter`, `RdfXmlWriter`) | Serialisation de graphes |

---

## 3. Premier graphe RDF : "Hello World" semantique

### Objectif

Nous allons creer un graphe RDF minimal contenant deux triplets, puis les afficher. Cela illustre les operations fondamentales :
1. Creer un graphe vide
2. Creer des noeuds (URI et literal)
3. Asserter des triplets
4. Parcourir et afficher les triplets

### Etape 1 : Creation d'un graphe vide

Un graphe RDF est un conteneur de triplets. La classe `Graph` est l'implementation standard de l'interface `IGraph` dans dotNetRDF.

In [ ]:
using VDS.RDF;

IGraph g = new Graph();

Console.WriteLine($"Graphe cree. Nombre de triplets : {g.Triples.Count}");

### Interpretation

A ce stade, nous avons cree un **graphe vide**. Un graphe RDF est une structure de donnees qui contiendra des triplets.

| Propriete | Valeur actuelle |
|-----------|-----------------|
| Nombre de triplets | 0 |
| URI de base | null (par defaut) |
| Espaces de noms | Predefinis (rdf, rdfs, xsd) |

Le graphe est pret a recevoir des triplets que nous allons ajouter dans la cellule suivante.

### Etape 2 : Creation de noeuds et assertion de triplets

Nous allons maintenant creer des noeuds et les relier par des triplets. Un triplet se compose de :
- **Sujet** : la ressource decrite (un noeud URI)
- **Predicat** : la relation/propriete (un noeud URI)
- **Objet** : la valeur ou ressource liee (noeud URI ou noeud literal)

In [ ]:
using VDS.RDF;
using System;

IGraph g = new Graph();

// Creation des noeuds URI (sujet et predicat)
IUriNode dotNetRDF = g.CreateUriNode(UriFactory.Create("http://www.dotnetrdf.org"));
IUriNode says = g.CreateUriNode(UriFactory.Create("http://example.org/says"));

// Creation des noeuds literals (objets)
ILiteralNode helloWorld = g.CreateLiteralNode("Hello World");
ILiteralNode bonjourMonde = g.CreateLiteralNode("Bonjour tout le Monde", "fr");

// Assertion des triplets dans le graphe
g.Assert(new Triple(dotNetRDF, says, helloWorld));
g.Assert(new Triple(dotNetRDF, says, bonjourMonde));

Console.WriteLine($"Nombre de triplets dans le graphe : {g.Triples.Count}");
Console.WriteLine();

// Affichage des triplets
foreach (Triple t in g.Triples)
{
    Console.WriteLine(t.ToString());
}

### Interpretation du "Hello World" RDF

Nous avons cree un graphe contenant **deux triplets** qui partagent le meme sujet et le meme predicat, mais different par l'objet :

| Triplet | Sujet | Predicat | Objet |
|---------|-------|----------|-------|
| 1 | `<http://www.dotnetrdf.org>` | `<http://example.org/says>` | `"Hello World"` |
| 2 | `<http://www.dotnetrdf.org>` | `<http://example.org/says>` | `"Bonjour tout le Monde"@fr` |

**Types de noeuds utilises** :

| Type | Interface | Exemple | Role |
|------|-----------|---------|------|
| URI Node | `IUriNode` | `http://www.dotnetrdf.org` | Identifie une ressource (sujet ou predicat) |
| Literal Node | `ILiteralNode` | `"Hello World"` | Valeur textuelle (objet) |
| Literal avec langue | `ILiteralNode` | `"Bonjour tout le Monde"@fr` | Valeur avec etiquette de langue |

> **Point cle** : La methode `g.Assert()` ajoute un triplet au graphe. La methode `UriFactory.Create()` interne les URIs pour optimiser la memoire et les comparaisons d'egalite.

> **Attention** : La sortie affichee par `ToString()` n'est pas une syntaxe RDF standard. C'est une representation de debogage. Pour une serialisation conforme, il faut utiliser un Writer (cf. SW-2).

---

## 4. Vue d'ensemble de la serie Semantic Web

Ce notebook est le premier d'une serie de **13 notebooks** progressifs organises en 4 parties, couvrant l'ensemble du spectre du Web semantique.

### Partie 1 : Fondations RDF (.NET C#)

| # | Notebook | Duree | Sujet |
|---|----------|-------|-------|
| 1 | **SW-1-Setup** (ce notebook) | 20 min | Installation, contexte, premier graphe |
| 2 | [SW-2-RDFBasics](SW-2-RDFBasics.ipynb) | 45 min | Triplets, types de noeuds, serialisation |
| 3 | [SW-3-GraphOperations](SW-3-GraphOperations.ipynb) | 50 min | Lecture, ecriture, fusion, listes RDF |
| 4 | [SW-4-SPARQL](SW-4-SPARQL.ipynb) | 45 min | Query Builder, SELECT, FILTER, OPTIONAL |

### Partie 2 : Donnees Liees et Ontologies (.NET C#)

| # | Notebook | Duree | Sujet |
|---|----------|-------|-------|
| 5 | [SW-5-LinkedData](SW-5-LinkedData.ipynb) | 50 min | DBpedia, Wikidata, requetes federees |
| 6 | [SW-6-RDFS](SW-6-RDFS.ipynb) | 40 min | Vocabulaire RDFS, inference, hierarchies |
| 7 | [SW-7-OWL](SW-7-OWL.ipynb) | 50 min | OWL 2, ontologies, raisonnement |

### Partie 3 : Ecosysteme Python et Standards Modernes

| # | Notebook | Duree | Sujet |
|---|----------|-------|-------|
| 8 | [SW-8-PythonRDF](SW-8-PythonRDF.ipynb) | 50 min | rdflib, SPARQLWrapper, comparaison .NET/Python |
| 9 | [SW-9-SHACL](SW-9-SHACL.ipynb) | 45 min | Validation pySHACL, qualite des donnees |
| 10 | [SW-10-JSONLD](SW-10-JSONLD.ipynb) | 40 min | JSON-LD, Schema.org, SEO semantique |
| 11 | [SW-11-RDFStar](SW-11-RDFStar.ipynb) | 40 min | RDF 1.2, quoted triples, SPARQL-Star |

### Partie 4 : Graphes de Connaissances et IA

| # | Notebook | Duree | Sujet |
|---|----------|-------|-------|
| 12 | [SW-12-KnowledgeGraphs](SW-12-KnowledgeGraphs.ipynb) | 55 min | Construction KGs, OWLReady2, visualisation |
| 13 | [SW-13-GraphRAG](SW-13-GraphRAG.ipynb) | 50 min | KG + LLMs, extraction d'entites, RAG |

> **Duree totale estimee** : environ 10 heures de travail pratique.

---

## Resume

| Element | Ce que nous avons appris |
|---------|-------------------------|
| Web semantique | Vision de Tim Berners-Lee, pile W3C, donnees comprehensibles par les machines |
| dotNetRDF | Bibliotheque C# pour RDF, installee via NuGet |
| Graphe RDF | Conteneur de triplets, cree avec `new Graph()` |
| Triplet | Sujet-Predicat-Objet, asserte avec `g.Assert()` |
| Noeuds | `IUriNode` (ressources), `ILiteralNode` (valeurs) |

### Prochaine etape

Dans le notebook suivant (**SW-2-RDFBasics**), nous approfondirons :
- Les differents types de noeuds RDF (URI, Blank, Literal type/langue)
- La serialisation dans plusieurs formats (NTriples, RDF/XML, Turtle)
- La lecture de fichiers RDF existants

---

**Navigation** : [Index](README.md) | [2-RDFBasics >>](SW-2-RDFBasics.ipynb)